In [ ]:
# hide
# default_exp handlers
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

# ASGIHandlers

In [ ]:
# export

from asgiref.sync import sync_to_async
from django.core.handlers.asgi import ASGIHandler


class AsyncFileASGIHandler(ASGIHandler):
    async def send_response(self, response, send):
        if response.is_async:
            return response.stream(send)
        else:
            return super().send_response(response, send)

<IPython.core.display.Javascript object>

## Tests

In [ ]:
class Response:
    is_async = True

    async def stream(self, send):
        await send("foobar")

<IPython.core.display.Javascript object>

In [ ]:
async def send(data):
    return data

<IPython.core.display.Javascript object>

In [ ]:
# app = AsyncFileASGIHandler()
# response = Response()
# app.send_response(response, send)

<IPython.core.display.Javascript object>